In [17]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import lightgbm as lgb
from sklearn import svm
import xgboost as xgb
import catboost
from sklearn.neural_network import MLPRegressor
import optuna

from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [18]:
def load_data(y):
    path = '../../data/std_data/'
    x_train_std = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).values
    x_test_std = pd.read_pickle(path +'test/{}_x.pkl'.format(str(y))).values
    y_train = pd.read_pickle(path +'train/{}_y.pkl'.format(str(y))).values
    y_test = pd.read_pickle(path +'test/{}_y.pkl'.format(str(y))).values
    features = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).columns
    return x_train_std, x_test_std, y_train, y_test, features

In [19]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [20]:
def train():
    
    # 予測した確率全体を格納
    probs_all_lr = np.array([])
    probs_all_lda = np.array([])
    probs_all_sv = np.array([])
    probs_all_lgbm = np.array([])
    probs_all_xgb = np.array([])
    probs_all_cb = np.array([])
    probs_all_mlp = np.array([])
    
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        # データの生成
        x_train_std, x_test_std, y_train, y_test, features = load_data(y)
        y_true_all = np.hstack((y_true_all, y_test))
       
        # logistic regression
        lr = LogisticRegression(class_weight="balanced", solver="liblinear",  penalty="l2", C=0.0001, random_state=0,) # ロジスティック回帰モデルのインスタンスを作成
        lr.fit(x_train_std, y_train) # ロジスティック回帰モデルの重みを学習
        probs_lr = lr.predict_proba(x_test_std)[:,1]
        probs_all_lr = np.hstack((probs_all_lr, probs_lr))
        
        # LDA
        lda = LDA(solver="eigen", shrinkage=1).fit(x_train_std,  y_train)
        probs_lda = lda.predict_proba(x_test_std)[:,1]
        probs_all_lda = np.hstack((probs_all_lda, probs_lda))
        
        # svm
        sv = svm.SVR(kernel="sigmoid",
                                     degree=4,
                                     gamma=0.043502212815589775,
                                     coef0=0.20190829020616494,
                                     tol=0.0001,
                                     C=0.000245786293391316,
                                     epsilon=0.3056167642389302,
                                    verbose=False,)
        sv.fit(x_train_std, y_train)
        probs_sv = sv.predict(x_test_std)
        probs_all_sv = np.hstack((probs_all_sv, probs_sv))
        
        # xgb
        xgboost = xgb.XGBRegressor(silent= True, 
                               random_state=0,
                               max_depth=4,
                               learning_rate=0.12765177534095626,
                               n_estimators = 46,
                               gamma=0.060805284848630535,
                               reg_lambda=4.995675788308118,
                               reg_alpha=2.1912254426545754,
                               sub_sample=0.45297631180790854,
                               scale_pos_weight=1.1672978934986058)
        xgboost.fit(x_train_std, y_train)
        probs_xgb = xgboost.predict(x_test_std)
        probs_all_xgb = np.hstack((probs_all_xgb, probs_xgb))
        
        
        
        # lgbm
        lgbm = lgb.LGBMRegressor(
            random_state=0,
            verbosity=-1,
            bagging_seed=0,
            boost_from_average='true',
            metric='auc',
            bagging_freq=4,
            min_data_in_leaf=21,
            max_depth=13,
            learning_rate=0.08731913651405197,
            n_estimators=3394,
            subsample=0.7054763057027115,
            num_leaves=438,
            reg_lambda=0.9377125325944119,  
        )
        
        lgbm.fit(x_train_std, y_train)
        probs_lgbm = lgbm.predict(x_test_std)
        probs_all_lgbm = np.hstack((probs_all_lgbm, probs_lgbm))
        
        # catboost
        cb = catboost.CatBoostRegressor(
            random_state=0,
            iterations=258,
            depth=2,
            learning_rate=0.019083573879517587,
            random_strength=84,
            bagging_temperature=0.3233702745357832,
            od_type="Iter",
            od_wait=32, 
            logging_level='Silent')
        cb.fit(x_train_std, y_train)
        probs_cb = cb.predict(x_test_std)
        probs_all_cb = np.hstack((probs_all_cb, probs_cb))   
        
        # mlp
        mlp = MLPRegressor(hidden_layer_sizes=(32,),
                           activation='relu',
                           solver='adam',
                           alpha=4.76324733221396,
                           batch_size='auto',
                           learning_rate='constant', 
                           learning_rate_init=0.0012043271455668674, 
                           power_t=0.5,
                           max_iter=1000, 
                           shuffle=True,
                           random_state=0, 
                           tol=0.0001, 
                           verbose=False, 
                           warm_start=False, 
                           momentum=0.9,
                           nesterovs_momentum=True, 
                           early_stopping=False, 
                           validation_fraction=0.1, 
                           beta_1=0.022158342014810775, 
                           beta_2= 0.7802116425099002,
                           epsilon=1e-08,
                           )
        mlp.fit(x_train_std, y_train)
        probs_mlp = mlp.predict(x_test_std)
        probs_all_mlp = np.hstack((probs_all_mlp, probs_mlp))   
    
    auc_lr = roc_auc_score(y_true_all, probs_all_lr)
    auc_lda = roc_auc_score(y_true_all, probs_all_lda)
    auc_sv = roc_auc_score(y_true_all, probs_all_sv)
    auc_xgb = roc_auc_score(y_true_all, probs_all_xgb)
    auc_lgbm = roc_auc_score(y_true_all, probs_all_lgbm)
    auc_cb = roc_auc_score(y_true_all, probs_all_cb)
    auc_mlp = roc_auc_score(y_true_all, probs_all_mlp)
    
    all_models_probs = [probs_all_lr, probs_all_lda, probs_all_sv, probs_all_xgb, probs_all_lgbm, probs_all_cb, probs_all_mlp]
    
    print("len: {0} ".format(len(y_true_all) ))

    print("AUC LR: ",auc_lr)
    print("AUC LDA: ",auc_lda)
    print("AUC svm: ",auc_sv)
    print("AUC xgb: ",auc_xgb)
    print("AUC lgbm: ", auc_lgbm)
    print("AUC CB: ",auc_cb)
    print("AUC MLP: ",auc_mlp)
    print()
    return y_true_all, all_models_probs

In [21]:
def optimize(trial, y_true_all, all_models_probs):
    
    
    w_lr =trial.suggest_uniform('w_lr', 0.0, 1.0)
    w_lda =0
    w_sv = trial.suggest_uniform('w_sv', 0.0, 1.0)
    w_xgb = trial.suggest_uniform('w_xgb', 0.0, 1.0)
    w_lgbm =trial.suggest_uniform('w_lgbm', 0.0, 1.0)
    w_cb =  0
    w_mlp = trial.suggest_uniform('w_mlp', 0.0, 1.0)
    
    # 各モデルの重み
    w = np.array([w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb, w_mlp])
    w_norm = w/np.sum(w)
    
    print("[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] = ", w)
    print("norm = ", w_norm)
    
    # 各モデル結果の重み付き平均
    probs_weighted_average = np.array([0 for i in range(len(y_true_all))], dtype='float64')
    for probs, weight in zip(all_models_probs, w_norm):
        probs_weighted_average += (probs * weight)

    auc = roc_auc_score(y_true_all, probs_weighted_average)
    
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_weighted_average)
    
#     plot_roc_curve(fpr, tpr, auc)

    print("AUC all: ",auc)
    print()
    return -auc

In [22]:
def main():
    np.set_printoptions(precision=3, suppress=True)
    
    y_true_all, all_models_probs = train()
    
    study = optuna.create_study()
    study.optimize(lambda trial: optimize(trial, y_true_all, all_models_probs), n_trials=10000)
    print(study.best_trial)

In [23]:
main()

len: 211 
AUC LR:  0.7482389405466329
AUC LDA:  0.7418991265145111
AUC svm:  0.794590025359256
AUC xgb:  0.7607072414764722
AUC lgbm:  0.7602141448295294
AUC CB:  0.7561284868977177
AUC MLP:  0.777120315581854

[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.156 0.    0.31  0.041 0.53  0.    0.846]
AUC all:  0.7768385460693152



[I 2019-08-24 12:07:14,152] Finished trial#0 resulted in value: -0.7768385460693152. Current best value is -0.7768385460693152 with parameters: {'w_lr': 0.155787181116115, 'w_sv': 0.30963060389190644, 'w_xgb': 0.04108205554267208, 'w_lgbm': 0.5296914906600672, 'w_mlp': 0.8464782906009629}.


[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.683 0.    0.162 0.012 0.642 0.    0.656]
AUC all:  0.7716258100873485



[I 2019-08-24 12:07:14,265] Finished trial#1 resulted in value: -0.7716258100873485. Current best value is -0.7768385460693152 with parameters: {'w_lr': 0.155787181116115, 'w_sv': 0.30963060389190644, 'w_xgb': 0.04108205554267208, 'w_lgbm': 0.5296914906600672, 'w_mlp': 0.8464782906009629}.


[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.406 0.    0.471 0.258 0.305 0.    0.807]
AUC all:  0.79149056072133



...

[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.13  0.    0.463 0.183 0.143 0.    0.782]
AUC all:  0.7923358692589462



[I 2019-08-24 12:30:05,446] Finished trial#1373 resulted in value: -0.7923358692589462. Current best value is -0.7944491406029868 with parameters: {'w_lr': 0.018735636670162738, 'w_sv': 0.609782869522277, 'w_xgb': 0.1257268888344056, 'w_lgbm': 0.10282314833705018, 'w_mlp': 0.6409335052733912}.


[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.061 0.    0.676 0.162 0.117 0.    0.713]
AUC all:  0.7927585235277541



[I 2019-08-24 12:30:07,282] Finished trial#1374 resulted in value: -0.7927585235277541. Current best value is -0.7944491406029868 with parameters: {'w_lr': 0.018735636670162738, 'w_sv': 0.609782869522277, 'w_xgb': 0.1257268888344056, 'w_lgbm': 0.10282314833705018, 'w_mlp': 0.6409335052733912}.


KeyboardInterrupt: 

In [ ]:
# best value is -0.7947309101155255 with parameters: {'w_lda': 0.09921014790481464, 'w_sv': 0.4530074587471793, 'w_xgb': 0.22971392450097775, 'w_lgbm': 0.2501321890556603, 'w_mlp': 0.926253219022292}.

#best value is -0.7948717948717948 with parameters: {'w_lr': 0.08360537746692473, 'w_sv': 0.7436098375618999, 'w_xgb': 0.22134975813644708, 'w_lgbm': 0.23431770706459376, 'w_mlp': 0.8118605353214242}.

# best value is -0.7971259509721048 with parameters: {'w_sv': 0.8002438174499535, 'w_xgb': 0.00031518275577795086, 'w_mlp': 0.0018017698441267613}.

#best value is -0.7976894899971824 with parameters: {'w_sv': 0.49087490720537474, 'w_mlp': 0.00212194974236059}.
